# Multiple Timeseries Forecasting with Nixtla

## Installing Requirements

Activate a Python environment and install the following

In [ ]:
%%capture
%pip install pandas
%pip install pandas
%pip install numpy
%pip install darts
%pip install matplotlib
%pip install gluonts[torch] optuna
%pip install ipykernel
%pip install --upgrade nbformat
%pip install lightgbm xgboost
%pip install seaborn
%pip install distributed
%pip install datasetsforecast
%pip install mlforecast
%pip install statsforecast
%pip install tqdm

## Import packages

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Dataset

In this tutorial, we will train and evaluate multiple time-series forecasting models using the [Store Item Demand Forecasting Challenge dataset from Kaggle](https://www.kaggle.com/competitions/demand-forecasting-kernels-only/data?select=train.csv). This dataset has 10 different stores and each store has 50 items, i.e. total of 500 daily level time series data for five years (2013–2017).

# Download data

* Download the train.csv from https://www.kaggle.com/competitions/demand-forecasting-kernels-only/data?select=train.csv.
* Create a `./data` directory inside the directory of this Python notebook
* Save the train.csv inside the `./data` directory

# Load data


In [ ]:
data = pd.read_csv("./data/train.csv", index_col=0)
data = data.reset_index()
data.head()

In [ ]:
print(f"The dataset has {data.shape[0]} rows and {data.shape[1]} columns")

# Data fields

* date - Date of the sale data. There are no holiday effects or store closures.
* store - Store ID
* item - Item ID
* sales - Number of items sold at a particular store on a particular date.

In [ ]:
data.describe()

* We don't have any negative "sales" values
* We need to convert "store" and "item" columns to type string

### Reduce the amount of data

For simplicity & speed, we will only keep data from the first 2 out of the 10 stores

In [ ]:
data = data.loc[data["store"] <= 2]
print(f"The dataset now has {data.shape[0]} rows and {data.shape[1]} columns")

### Convert "store" and "item" columns to type string

In [ ]:
data[["store", "item"]] = data[["store", "item"]].astype(str)

### Check for duplicate rows

In [ ]:
count_duplicate_rows = len(data)-len(data.drop_duplicates())
print(f"There are {count_duplicate_rows} duplicate rows")

### Create a new column by concatenating "store" and "item" columns

In [ ]:
data["store_item"] = data["store"] + "_" + data["item"]

# Convert date column to datetime

In [ ]:
data["date"] = pd.to_datetime(data["date"]) 

### Calculate total sales per date, store and item

In [ ]:
item_sales_per_date = data.groupby(["date","store_item"])["sales"].aggregate("sum")
item_sales_per_date = item_sales_per_date.reset_index()
item_sales_per_date.columns = ["date","store_item","sales"]
item_sales_per_date = item_sales_per_date.sort_values("date", ascending=True)
item_sales_per_date.head()
item_sales_per_date.tail()

### Plot total sales for all products over time

In [ ]:
total_sales_per_date = item_sales_per_date.groupby(["date"])["sales"].aggregate("sum")
total_sales_per_date.plot()

## Not all items sell every day

* For each "store_item" value add all the missing "date" values and fill in "sales" with 0s
## Generate an index with all combinations of "date" and "store_item" values

In [ ]:
multiindex = list(zip(item_sales_per_date["date"], item_sales_per_date["store_item"]))
multiindex = pd.MultiIndex.from_tuples(multiindex, names=('index_1', 'index_2'))
dataset = item_sales_per_date.copy()
dataset.index = multiindex

In [ ]:
import itertools

idx_dates = list(pd.date_range(min(item_sales_per_date["date"]), max(item_sales_per_date["date"])))
idx_ids = list(dataset["store_item"].unique())
idx = list(itertools.product(idx_dates, idx_ids))
dataset = dataset.reindex(index=idx, columns=None)
dataset = dataset.reset_index()
dataset.head()

### For each "store_item" value fill add all the missing "date" values and fill in "sales" with 0s

In [ ]:
# Fill missing values in 'date' column with values from 'index_1' column
dataset["date"].fillna(dataset["index_1"], inplace=True)

# Fill missing values in 'store_item' column with values from 'index_2' column
dataset["store_item"].fillna(dataset["index_2"], inplace=True)

# Fill missing values in 'sales' column with 0
dataset["sales"].fillna(0, inplace=True)

# Drop 'index_1' and 'index_2' columns
dataset.drop(columns=["index_1", "index_2"], inplace=True)

# Set 'date' column as the index and rename index to "date"
dataset.set_index("date", inplace=True)
dataset.index.name = "date"

# Display the first few rows of the DataFrame
dataset.head()

# Create the dataset that we will use for training the forecasting models

In [ ]:
# Create the dataset that we will use for training the forecasting models
# Rename columns to match the Nixtlaverse's expectations
# The 'store_item' becomes 'unique_id' representing the unique identifier of the time series
# The 'date' becomes 'ds' representing the time stamp of the data points
# The 'sales' becomes 'y' representing the target variable we want to forecast

# Select the entire dataset for training
Y_df = dataset.copy()

# Reset the index to have a default integer index
Y_df.reset_index(inplace=True)

# Rename columns according to the requirements
Y_df.rename(columns={'store_item': 'unique_id', 'date': 'ds', 'sales': 'y'}, inplace=True)

# Convert the 'y' column to integer type
Y_df['y'] = Y_df['y'].astype(int)

# Convert the 'unique_id' column to string type
Y_df['unique_id'] = Y_df['unique_id'].astype(str)

# Convert the 'ds' column to datetime format to ensure proper handling of date-related operations in subsequent steps
Y_df['ds'] = pd.to_datetime(Y_df['ds'])

# Display the last few rows of the DataFrame to verify the changes
Y_df.tail()


In [ ]:
from tqdm.autonotebook import tqdm
from statsforecast import StatsForecast
# Feature: plot random series for EDA
StatsForecast.plot(Y_df)

# Check for seasonality in the total number of 'sales' per 'date'

In [ ]:
import darts
from darts import TimeSeries

seasonality_check_data = total_sales_per_date.reset_index()
seasonality_check_data = seasonality_check_data.set_index("date")
seasonality_check_data_series = TimeSeries.from_times_and_values(seasonality_check_data.index, seasonality_check_data["sales"].values)

# Autocorrelation function 
Autocorrelation ([source](https://www.investopedia.com/terms/a/autocorrelation.asp)) is a mathematical representation of the degree of similarity between a given time series and a lagged version of itself over successive time intervals. It's conceptually similar to the correlation between two different time series, but autocorrelation uses the same time series twice: once in its original form and once lagged one or more time periods. 

* Autocorrelation represents the degree of similarity between a given time series and a lagged version of itself over successive time intervals.
* Autocorrelation measures the relationship between a variable's current value and its past values.
* An autocorrelation of +1 represents a perfect positive correlation, while an autocorrelation of -1 represents a perfect negative correlation.

The plot_acf function is called with the following parameters:

* **seasonality_check_data_series**: This is the time series data for which the autocorrelation function will be plotted.
* **m=7**: This parameter specifies the period for seasonal decomposition. Here, m=7 indicates a weekly seasonality trend.
* **alpha=0.05**: This parameter determines the significance level for the confidence interval. A value of 0.05 corresponds to a 95% confidence level.
* **max_lag=30**: This parameter sets the maximum lag to consider in the autocorrelation function plot.

In [ ]:
from darts.utils.statistics import plot_acf

plot_acf(seasonality_check_data_series, m=7, alpha=0.05, max_lag=30)

The ACF presents a spike at x in [1, 7, 14, 21], which suggests a weekly seasonality trend (highlighted). The blue zone determines the significance of the statistics for a confidence level of $\alpha = 5\%$. We can also run a statistical check of seasonality for each candidate period `m`.

The confidence interval of the autocorrelation function provides a range of values within which the true autocorrelation of a time series is likely to fall.

In simple terms, imagine you have a set of data points that represent measurements taken over time. The autocorrelation function tells you how related each data point is to the ones that came before it. The confidence interval gives you an idea of how reliable that measurement of autocorrelation is.

The confidence interval is expressed as a range of values around the calculated autocorrelation. For example, if the autocorrelation of a specific lag (time gap between observations) is 0.7 and the confidence interval is ±0.1, it means we are reasonably confident that the true autocorrelation lies between 0.6 and 0.8.

In [ ]:
""" By setting max_lag=12*30, you're considering a maximum lag of 12 months, which allows you to analyze the autocorrelation over
    a longer time period, potentially capturing seasonal patterns or long-term trends in the data. 
"""
plot_acf(seasonality_check_data_series, m=7, alpha=0.05, max_lag=12*30)

We will now train multiple Statistical & ML models and evaluate which one performs best

## Create forecasts with Stats & ML methods.

# Stats Methods with StatsForecast

In [ ]:
# Import necessary models from the statsforecast library
from statsforecast.models import (
    # ADIDA: Adaptive combination of Intermittent Demand Approaches, a model designed for intermittent demand
    ADIDA,
    # AutoETS: Automated Exponential Smoothing model that automatically selects the best Exponential Smoothing model based on AIC
    AutoETS,
    # CrostonOptimized: A model specifically designed for intermittent demand forecasting
    CrostonOptimized,
    # HistoricAverage: This model uses the average of all historical data as the forecast
    HistoricAverage,
    # IMAPA: Intermittent Multiplicative AutoRegressive Average, a model for intermittent series that incorporates autocorrelation
    IMAPA,
    # Naive: A simple model that uses the last observed value as the forecast
    Naive,
    # SeasonalNaive: A model that uses the previous season's data as the forecast
    SeasonalNaive,
)

# Define the number of days in the future for which we will make a forecast
horizon = 30

# Define the length of the seasonality window (e.g., 7 days for weekly seasonality)
season_length = 7

# Define the number of days that the model will use to make a forecast (e.g., 6 months)
window_size = 6 * 30

# Define a list of forecasting models to be used
models = [
    # SeasonalNaive: A model that uses the previous season's data as the forecast
    SeasonalNaive(season_length=season_length),
    
    # Naive: A simple model that uses the last observed value as the forecast
    Naive(),
    
    # HistoricAverage: This model uses the average of all historical data as the forecast
    HistoricAverage(),
    
    # CrostonOptimized: A model specifically designed for intermittent demand forecasting
    CrostonOptimized(),
    
    # ADIDA: Adaptive combination of Intermittent Demand Approaches, a model designed for intermittent demand
    ADIDA(),
    
    # IMAPA: Intermittent Multiplicative AutoRegressive Average, a model for intermittent series that incorporates autocorrelation
    IMAPA(),
    
    # AutoETS: Automated Exponential Smoothing model that automatically selects the best Exponential Smoothing model based on AIC
    AutoETS(season_length=season_length)
]


In [ ]:
# Instantiate the StatsForecast class for forecasting
sf = StatsForecast(
    models=models,  # A list of forecasting models to be used
    freq='D',       # The frequency of the time series data ('D' stands for daily frequency)
    n_jobs=-1,      # The number of CPU cores to use for parallel execution (-1 means use all available cores)
)

In [ ]:
from time import time
from statsforecast.utils import ConformalIntervals

# Create an instance of ConformalIntervals with the desired parameters
prediction_intervals = ConformalIntervals()

# Get the current time before forecasting starts, this will be used to measure the execution time
init = time()

# Call the forecast method of the StatsForecast instance to predict the next horizon days
# Level is set to [90], which means that it will compute the 90% prediction interval
# prediction_intervals is set to True to compute prediction intervals
fcst_df = sf.forecast(df=Y_df, h=horizon, level=[90], prediction_intervals=prediction_intervals)

# Get the current time after the forecasting ends
end = time()

# Calculate and print the total time taken for the forecasting in minutes
print(f'Forecast Minutes: {(end - init) / 60}')


In [ ]:
fcst_df.head()

# ML Methods with MLForecast

In [ ]:
from mlforecast import MLForecast
from mlforecast.target_transforms import Differences
from mlforecast.utils import PredictionIntervals
from window_ops.expanding import expanding_mean

In [ ]:
# Import the necessary models from various libraries

# LGBMRegressor: A gradient boosting framework that uses tree-based learning algorithms from the LightGBM library
from lightgbm import LGBMRegressor

# XGBRegressor: A gradient boosting regressor model from the XGBoost library
from xgboost import XGBRegressor

# LinearRegression: A simple linear regression model from the scikit-learn library
from sklearn.linear_model import LinearRegression

In [ ]:
from mlforecast import MLForecast
from mlforecast.target_transforms import Differences
from mlforecast.utils import PredictionIntervals
from window_ops.expanding import expanding_mean
# Instantiate the MLForecast object
mlf = MLForecast(
    models=[LGBMRegressor(max_depth=10), XGBRegressor(max_depth=10, eval_metric='rmse'), LinearRegression()],  # List of models for forecasting: LightGBM, XGBoost and Linear Regression
    freq='D',  # Frequency of the data - 'D' for daily frequency
    lags=list(range(1, 7)),  # Specific lags to use as regressors: 1 to 6 days
    lag_transforms = {
        1:  [expanding_mean],  # Apply expanding mean transformation to the lag of 1 day
    },
    date_features=['year', 'month', 'day', 'dayofweek', 'quarter', 'week', 'dayofyear'],  # Date features to use as regressors
    target_transforms=[Differences([1])],


)

In [ ]:
# Start the timer to calculate the time taken for fitting the models
init = time()

# Fit the MLForecast models to the data, with prediction intervals set using a window size of window_size days
mlf.fit(Y_df,  prediction_intervals=PredictionIntervals(n_windows=10, h=48))

# Calculate the end time after fitting the models
end = time()

# Print the time taken to fit the MLForecast models, in minutes
print(f'MLForecast Minutes: {(end - init) / 60}')

In [ ]:
fcst_mlf_df = mlf.predict(h=horizon, level=[90])
fcst_mlf_df.head()

# Forecast plots 

Per store, item and forecast model

In [ ]:
fcst_df = fcst_df.merge(fcst_mlf_df, how='left', on=['unique_id', 'ds'])

In [ ]:
sf.plot(Y_df, fcst_df, max_insample_length=30)

# Validate Model’s Performance

## Cross Validation in StatsForecast

In [ ]:
init = time()
cv_df = sf.cross_validation(df=Y_df, h=horizon, n_windows=3, step_size=horizon, level=[90])
end = time()
print(f'CV Minutes: {(end - init) / 60}')

In [ ]:
init = time()
cv_mlf_df = mlf.cross_validation(
    df=Y_df, 
    h=horizon, 
    n_windows=3, 
    step_size=horizon, 
    level=[90],
)
end = time()
print(f'CV Minutes: {(end - init) / 60}')

# Plot CV

In [ ]:
cv_df = cv_df.merge(cv_mlf_df.drop(columns=['y']), how='left', on=['unique_id', 'ds', 'cutoff'])

In [ ]:
cv_df.head()

In [ ]:
cutoffs = cv_df['cutoff'].unique()

## Visualize cross validation splits for a specific unique_id

In [ ]:
for cutoff in cutoffs:
    img = sf.plot(
        Y_df, 
        cv_df.query('cutoff == @cutoff').drop(columns=['y', 'cutoff']), 
        max_insample_length=30*4, 
        unique_ids=['1_1'],
    )

# Aggregate sales forecasts for all items

In [ ]:
agg_cv_df = cv_df.loc[:,~cv_df.columns.str.contains('hi|lo')].groupby(['ds', 'cutoff']).sum(numeric_only=True).reset_index()
agg_cv_df.insert(0, 'unique_id', 'Total sales (all items)')

In [ ]:
agg_Y_df = Y_df.groupby(['ds']).sum(numeric_only=True).reset_index()
agg_Y_df.insert(0, 'unique_id', 'Total sales (all items)')

In [ ]:
for cutoff in cutoffs:
    img = sf.plot(
        agg_Y_df, 
        agg_cv_df.query('cutoff == @cutoff').drop(columns=['y', 'cutoff']),
        max_insample_length=30*4,
    )

# Evalute forecasts per store_item and CV window

In [ ]:
from typing import List, Callable

from distributed import Client
from fugue import transform
from fugue_dask import DaskExecutionEngine
from datasetsforecast.losses import mse, mae, smape, mape

In [ ]:
def evaluate(df: pd.DataFrame, metrics: List[Callable]) -> pd.DataFrame:
    eval_ = {}
    models = df.loc[:, ~df.columns.str.contains('unique_id|y|ds|cutoff|lo|hi')].columns
    for model in models:
        eval_[model] = {}
        for metric in metrics:
            eval_[model][metric.__name__] = metric(df['y'], df[model])
    eval_df = pd.DataFrame(eval_).rename_axis('metric').reset_index()
    eval_df.insert(0, 'cutoff', df['cutoff'].iloc[0])
    eval_df.insert(0, 'unique_id', df['unique_id'].iloc[0])
    return eval_df

In [ ]:
str_models = cv_df.loc[:, ~cv_df.columns.str.contains('unique_id|y|ds|cutoff|lo|hi')].columns
str_models = ','.join([f"{model}:float" for model in str_models])
cv_df['cutoff'] = cv_df['cutoff'].astype(str)
cv_df = cv_df.reset_index()
cv_df['unique_id'] = cv_df['unique_id'].astype(str)

Let’s create a dask client.

In [ ]:
client = Client() # without this, dask is not in distributed mode
# fugue.dask.dataframe.default.partitions determines the default partitions for a new DaskDataFrame
engine = DaskExecutionEngine({"fugue.dask.dataframe.default.partitions": 96})

The transform function takes the evaluate functions and applies it to each combination of time series (unique_id) and cross validation window (cutoff) using the dask client we created before.


In [ ]:
evaluation_df = transform(
    cv_df.loc[:, ~cv_df.columns.str.contains('lo|hi')], 
    evaluate, 
    engine="dask",
    params={'metrics': [mse, mae, mape, smape]}, 
    schema=f"unique_id:str,cutoff:str,metric:str, {str_models}", 
    as_local=True,
    partition={'by': ['unique_id', 'cutoff']}
)

In [ ]:
evaluation_df.head()

In [ ]:
# Calculate the mean metric for each cross validation window
evaluation_df.groupby(['cutoff', 'metric']).mean(numeric_only=True)

# Distribution of errors

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
evaluation_df_melted = pd.melt(evaluation_df, id_vars=['unique_id', 'cutoff', 'metric'], var_name='model', value_name='error')
evaluation_df_melted

## Distribution of SMAPE errors per model, unique_id and evaluation metric

In [ ]:
sns.violinplot(evaluation_df_melted.query('metric=="mse"'), x='error', y='model').set_title('Distribution of MSE errors')

## Distribution of SMAPE errors per model, unique_id and evaluation metric

In [ ]:
sns.violinplot(evaluation_df_melted.query('metric=="smape"'), x='error', y='model').set_title('Distribution of SMAPE errors')

In [ ]:
trained_models = list(evaluation_df_melted.model.unique())

# Choose models for groups of series

We can find the best performing model per store_item and evaluation metric.
In how many cross validation fold & metric is each model overperforming the rest?

In [ ]:
# Choose the best model for each time series, metric, and cross validation window
evaluation_df['best_model'] = evaluation_df.idxmin(axis=1, numeric_only=True)
# count how many times a model wins per metric and cross validation window
count_best_model = evaluation_df.groupby(['unique_id', 'metric', 'best_model']).size().rename('n').to_frame().reset_index()
# plot results
sns.barplot(count_best_model, x='n', y='best_model', hue='metric')

AutoETS is the best performing model for all evaluation metrics

This does not mean that AutoETS is the best performing model for each individual "store_item"

In [ ]:
count_best_model

What is the best model for store_item="1_1" sales forecasting?

In [ ]:
# For the mse, calculate how many times a model wins
# add colors
colors = ['#ff9999','#66b3ff','#99ff99','#ffcc99']
count_best_model_mse = count_best_model.query('metric == "mse" & unique_id == "1_1"')
counts_series_mse = count_best_model_mse["n"]
plt.pie(counts_series_mse, labels=count_best_model_mse["best_model"], autopct='%.0f%%', colors=colors)
plt.title("Winning Models Based On MSE - Store_item='1_1'")
plt.show()

In [ ]:
# For the mse, calculate how many times a model wins
count_best_model_mape = count_best_model.query('metric == "mape" & unique_id == "1_1"')
counts_series_mape = count_best_model_mape["n"]
plt.pie(counts_series_mape, labels=count_best_model_mape["best_model"], autopct='%.0f%%', colors=colors)
plt.title("Winning Models Per CV Fold Based On MAPE - Store_item='1_1'")
plt.show()

XGBRegressor was the best performing model based on MSE for 2 out of the 3 validation folds of store_item 1_1.

In [ ]:
# For the mse, calculate how many times a model wins
count_best_model_smape = count_best_model.query('metric == "smape" & unique_id == "1_1"')
counts_series_smape = count_best_model_smape["n"]
plt.pie(counts_series_smape, labels=count_best_model_smape["best_model"], autopct='%.0f%%', colors=colors)
plt.title("Winning Models Per CV Fold Based On SMAPE - Store_item='1_1'")
plt.show()

LGBMRegressor was the best performing model based on MSE for 2 out of the 3 validation folds of store_item 1_1.

Visualize the forecasts of the best model for unique_id == "1_1"

* The best model based on MAPE is XGBRegressor
* The best model based on SMAPE is LGBMRegressor

In [ ]:
eval_series_df = evaluation_df.query('metric == "mape" & unique_id == "1_1"').groupby(['unique_id']).mean(numeric_only=True)
eval_series_df['best_model'] = eval_series_df.idxmin(axis=1)

sf.plot(Y_df, cv_df.drop(columns=['cutoff', 'y']), 
        max_insample_length=30 * 2, 
        models=['XGBRegressor'],
        unique_ids=eval_series_df.query('best_model == "XGBRegressor"').index[:8])

In [ ]:
eval_series_df = evaluation_df.query('metric == "smape" & unique_id == "1_1"').groupby(['unique_id']).mean(numeric_only=True)
eval_series_df['best_model'] = eval_series_df.idxmin(axis=1)

sf.plot(Y_df, cv_df.drop(columns=['cutoff', 'y']), 
        max_insample_length=30 * 2, 
        models=['LGBMRegressor'],
        unique_ids=eval_series_df.query('best_model == "LGBMRegressor"').index[:8])

Visualize the AutoETS forecasts for more unique_ids

In [ ]:
eval_series_df = evaluation_df.query('metric == "mape"').groupby(['unique_id']).mean(numeric_only=True)
eval_series_df['best_model'] = eval_series_df.idxmin(axis=1)

sf.plot(Y_df, cv_df.drop(columns=['cutoff', 'y']), 
        max_insample_length=30 * 2, 
        models=['AutoETS'],
        unique_ids=eval_series_df.query('best_model == "AutoETS"').index[:8])

Visualize the XGBRegressor forecasts for more unique_ids

In [ ]:
eval_series_df = evaluation_df.query('metric == "mape"').groupby(['unique_id']).mean(numeric_only=True)
eval_series_df['best_model'] = eval_series_df.idxmin(axis=1)

sf.plot(Y_df, cv_df.drop(columns=['cutoff', 'y']), 
        max_insample_length=30 * 2, 
        models=['XGBRegressor'],
        unique_ids=eval_series_df.query('best_model == "XGBRegressor"').index[:8])

# Sources

This code is based on the following publicly available resources
* [Nixtla Statistical, Machine Learning and Neural Forecasting methods](https://nixtla.github.io/statsforecast/docs/tutorials/statisticalneuralmethods.html)
* [Intro to Forecasting with Darts](https://github.com/unit8co/darts/blob/master/examples/00-quickstart.ipynb)
* [Store Item Demand Forecasting Challenge dataset from Kaggle](https://www.kaggle.com/competitions/demand-forecasting-kernels-only/data?select=train.csv) 